In [ ]:
import shutil
import os
import numpy as np
import dask
import dask.array as da
from dask.diagnostics import ProgressBar, ResourceProfiler
from dask.distributed import Client

from ddm.data_handling import read_file
from ddm.processing import ddm_dask
from ddm.fitting import findMeanSqFFT_np, computeAB
from ddm.plotting import plotSingleExpFit, plotISF

In [ ]:
filename = '../data/test_dataset_drift.tif'
ddmData = read_file(filename, delayed=True, chunk_size=25)

In [ ]:
ddmData

In [ ]:
def calc_ddm(img_fft, tau):
    img_diff = img_fft[:-tau, :, :] - img_fft[tau:, :, :]
    img_fft_sq = da.abs(img_diff) ** 2
    img_sum = da.sum(img_fft_sq, axis=0)
    return da.fft.fftshift(img_sum)

def calc_ddm_numpy(img_fft, tau):
    img_diff = img_fft[:-tau, :, :] - img_fft[tau:, :, :]
    img_fft_sq = np.abs(img_diff) ** 2
    img_sum = np.sum(img_fft_sq, axis=0)
    return np.fft.fftshift(img_sum)

In [ ]:
def get_taus(index, tau_step):
    if index == 0:
        return np.arange(1, (index + 1)*tau_step)
    else:
        return np.arange(index*tau_step, (index + 1)*tau_step)

In [ ]:
%%time
tau_step = 10
with ProgressBar():
    out = []
    img_fft = da.fft.fft2(ddmData.data).astype(np.complex64)
    for index in range(10):
        result = [calc_ddm_numpy(img_fft, tau) for tau in get_taus(index, tau_step)]
        out = dask.compute(result, scheduler="threads")

# out = np.asarray(out)

In [ ]:
out.shape

### Refactored

In [ ]:
%%time
taus = da.arange(1,10)
ddmMatrix = ddm_dask(ddmData.data, taus)